In [1]:
import json
import os
import _jsonnet
import os
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [2]:
from seq2struct.commands.infer import Inferer
from seq2struct.datasets.spider import SpiderItem
from seq2struct.utils import registry

In [3]:
import torch

In [149]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [4]:
# Define a function to load JSON data from a file
def load_json(filename):
    # Open the specified file in read mode, specifying utf-8 encoding
    with open(filename, 'r', encoding='utf-8') as file:
        # Load the JSON data from the file into a Python dictionary
        data = json.load(file)
    # Return the loaded data
    return data

In [5]:
def createPath(lr, bert_lr, end_lr, bs, att):
    lr_s = '%0.1e' % lr
    bert_lr_s = '%0.1e' % bert_lr
    end_lr_s = '0e0' if end_lr == 0 else '%0.1e' % end_lr

    base_bert_enc_size = 768
    enc_size = base_bert_enc_size

    model_name = '/bs=%(bs)d,lr=%(lr)s,bert_lr=%(bert_lr)s,end_lr=%(end_lr)s,att=%(att)d' % {
        'bs': bs,
        'lr': lr_s,
        'bert_lr': bert_lr_s,
        'end_lr': end_lr_s,
        'att': att
    }
    return model_name

In [6]:
# exp_config = json.loads(_jsonnet.evaluate_file("experiments/spider-configs/gap-run.jsonnet"))
exp_config = json.loads(_jsonnet.evaluate_file("experiments/spider-configs/spider-mT5-base-en-train_en_eval.jsonnet"))

In [7]:
model_config_path = exp_config["model_config"]
model_config_args = exp_config.get("model_config_args")

#print(model_config_path)
#print(model_config_args)

In [8]:
infer_config = json.loads(
    _jsonnet.evaluate_file(
        model_config_path, 
        tla_codes={'args': json.dumps(model_config_args)}))

#print(infer_config['model']['encoder_preproc'])

In [9]:
db_path = infer_config["model"]["encoder_preproc"]["db_path"]

db_path_for_json = infer_config['data']['val']['tables_paths'][0]

In [10]:
#db_path_for_json

In [11]:
inferer = Inferer(infer_config)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}


SpiderEncoderT5Preproc Model: google/mt5-base
No GAP - Generation-Augmented Pre-Training
T5 load Model: google/mt5-base


In [12]:
# model_dir = exp_config["logdir"] + "/bs=12,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1"
# checkpoint_step = exp_config["eval_steps"][0]

# '/home/studio-lab-user/gr-spider/Database-Systems/gap-text2sql/mrat-sql-gap/logdir/mT5-base-en-train/bs=4,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1/val_loss.txt'
logdir_path = exp_config["logdir"]
path_save = createPath(model_config_args['lr'], model_config_args['bert_lr'], model_config_args['end_lr'], model_config_args['bs'], model_config_args['att'])
model_dir = logdir_path + path_save

checkpoint_step = exp_config["eval_steps"][0]

In [13]:
model = inferer.load_model(model_dir, checkpoint_step)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'save_path': 'data/spider-en/mT5-base-nl2code-1115,output_from=true,fs=2,emb=t5,cvlink', 'use_seq_elem_rules': True}, 'encoder_preproc': {'compute_cv_link': True, 'compute_sc_link': True, 'db_path': 'data/spider-en/database', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'pretrained_checkpoint': 'models/mt5-base/pretrained_checkpoint/pytorch_model.bin', 'save_path': 'data/spider-en/mT5-base-nl2code-1115,output_from=true,fs=2,emb=t5,cvlink', 't5_version': 'google/mt5-base'}}


SpiderEncoderT5 Model: google/mt5-base
No GAP - Generation-Augmented Pre-Training


Some weights of the model checkpoint at google/mt5-base were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Parameter containing:
tensor([[ 1.7500e+00, -1.6719e+00,  2.4062e+00,  ...,  6.9580e-03,
         -9.8828e-01, -4.6875e-01],
        [ 8.5625e+00,  5.5625e+00, -1.7109e+00,  ...,  7.7812e+00,
         -5.2812e+00, -3.2188e+00],
        [ 6.8750e-01, -4.5312e-01,  5.7812e-01,  ...,  7.3828e-01,
         -3.0078e-01,  2.0312e-01],
        ...,
        [ 8.0859e-01, -1.2812e+00,  2.2344e+00,  ...,  8.0469e-01,
          7.7344e-01,  1.8672e+00],
        [ 2.9688e-01, -1.4453e+00,  9.6484e-01,  ...,  8.9453e-01,
          2.3906e+00, -4.1797e-01],
        [ 1.0469e+00, -2.7930e-01,  9.7266e-01,  ..., -3.1250e+00,
          1.2344e+00, -7.6562e-01]], requires_grad=True)
Parameter containing:
tensor([[ 1.7500e+00, -1.6719e+00,  2.4062e+00,  ...,  6.9580e-03,
         -9.8828e-01, -4.6875e-01],
        [ 8.5625e+00,  5.5625e+00, -1.7109e+00,  ...,  7.7812e+00,
         -5.2812e+00, -3.2188e+00],
        [ 6.8750e-01, -4.5312e-01,  5.7812e-01,  ...,  7.3828e-01,
         -3.0078e-01,  2.0312e-

In [14]:
from seq2struct.datasets.spider_lib.preprocess.get_tables import dump_db_json_schema
from seq2struct.datasets.spider import load_tables_from_schema_dict

In [15]:
def get_schema_dict_by_db_id(list_of_schemas_dicts, db_id):
    return [schema_info for schema_info in list_of_schemas_dicts if db_id in schema_info['db_id']][0]

In [66]:
def create_dfs_from_schema(schema):
    # Create DataFrame for table names
    table_names_df = pd.DataFrame({
        'table_names_original': schema['table_names_original'],
        'table_names': schema['table_names']
    })

    # Map table indices to table names
    table_index_to_name = {index: name for index, name in enumerate(schema['table_names'])}
    
    # Create DataFrame for column names with table names
    column_data = []
    for col_index, (table_index, col_name) in enumerate(schema['column_names']):
        table_name = table_index_to_name[table_index] if table_index != -1 else 'N/A'
        is_primary_key = col_index in schema['primary_keys']
        column_data.append((col_index, table_name, schema['column_names_original'][col_index][1], col_name, schema['column_types'][col_index], is_primary_key))

    column_names_df = pd.DataFrame(column_data, columns=['column_index', 'table_name', 'column_names_original', 'column_names', 'column_types', 'is_primary_key'])
    
    column_index_to_name = [name for index, name in schema['column_names']]

    # Create DataFrame for primary keys
    primary_keys_df = pd.DataFrame({
    'table_name': [table_index_to_name[schema['column_names'][x][0]] for x in schema['primary_keys']],
    'primary_key_column_index': schema['primary_keys'],
    'primary_key': [column_index_to_name[x] for x in schema['primary_keys']]
    })

    
    column_index_to_id = {index: name for index, name in enumerate(schema['column_names'])}
    
    foreign_keys_df = pd.DataFrame({
    'foreign_key_column': [x[0] for x in schema['foreign_keys']],
    'referenced_column': [x[1] for x in schema['foreign_keys']],
    'foreign_key_table': [table_index_to_name[column_index_to_id[x[0]][0]] for x in schema['foreign_keys']],
    'foreign_key_column_name': [column_index_to_id[x[0]][1] for x in schema['foreign_keys']],
    'referenced_table': [table_index_to_name[column_index_to_id[x[1]][0]] for x in schema['foreign_keys']],
    'referenced_column_name': [column_index_to_id[x[1]][1] for x in schema['foreign_keys']]
    }) 

    return table_names_df, column_names_df, primary_keys_df, foreign_keys_df

In [24]:
def infer(question, spider_schema):
    data_item = SpiderItem(
            text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
            code=None,
            schema=spider_schema,
            orig_schema=spider_schema.orig,
            orig={"question": question}
        )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        output = inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)
    return output[0]["inferred_code"]

In [18]:
schemas = os.listdir(db_path)

schemas_list = load_json(db_path_for_json)

In [19]:
import random
N=len(schemas)

selected_schemas = random.choices(schemas, k=N)
print('You can choose a database from the below:')
print(', '.join(selected_schemas))

You can choose a database from the below:
school_player, network_1, program_share, solvency_ii, employee_hire_evaluation, inn_1, game_1, scholar, employee_hire_evaluation, gymnast, game_injury, protein_institute, film_rank, flight_2, allergy_1, product_catalog, cre_Theme_park, cre_Doc_Control_Systems, chinook_1, club_1, academic, gymnast, club_1, real_estate_properties, hr_1, local_govt_in_alabama, department_store, swimming, book_2, restaurant_1, driving_school, manufactory_1, scientist_1, shop_membership, epinions_1, customers_card_transactions, entrepreneur, baseball_1, customers_campaigns_ecommerce, wrestler, election, store_product, network_2, chinook_1, icfp_1, game_1, gymnast, cre_Docs_and_Epenses, school_bus, concert_singer, local_govt_mdm, cre_Doc_Template_Mgt, baseball_1, election_representative, customers_campaigns_ecommerce, employee_hire_evaluation, csu_1, behavior_monitoring, storm_record, restaurants, customers_campaigns_ecommerce, college_1, dog_kennels, customers_campa

In [125]:
db_id = 'pilot_record'

In [126]:
#selected_schema = dump_db_json_schema("{path}/{db_id}/{db_id}.sqlite".format(path=db_path, db_id=db_id), db_id)
selected_schema = get_schema_dict_by_db_id(schemas_list, db_id)

In [127]:
#from seq2struct.utils.api_utils import refine_schema_names
# If you want to change your schema name, then run this; Otherwise you can skip this.
#refine_schema_names(my_schema)

In [128]:
selected_schema

{'column_names': [[-1, '*'],
  [0, 'aircraft id'],
  [0, 'order year'],
  [0, 'manufacturer'],
  [0, 'model'],
  [0, 'fleet series'],
  [0, 'powertrain'],
  [0, 'fuel propulsion'],
  [1, 'pilot id'],
  [1, 'pilot name'],
  [1, 'rank'],
  [1, 'age'],
  [1, 'nationality'],
  [1, 'position'],
  [1, 'join year'],
  [1, 'team'],
  [2, 'record id'],
  [2, 'pilot id'],
  [2, 'aircraft id'],
  [2, 'date']],
 'column_names_original': [[-1, '*'],
  [0, 'Aircraft_ID'],
  [0, 'Order_Year'],
  [0, 'Manufacturer'],
  [0, 'Model'],
  [0, 'Fleet_Series'],
  [0, 'Powertrain'],
  [0, 'Fuel_Propulsion'],
  [1, 'Pilot_ID'],
  [1, 'Pilot_name'],
  [1, 'Rank'],
  [1, 'Age'],
  [1, 'Nationality'],
  [1, 'Position'],
  [1, 'Join_Year'],
  [1, 'Team'],
  [2, 'Record_ID'],
  [2, 'Pilot_ID'],
  [2, 'Aircraft_ID'],
  [2, 'Date']],
 'column_types': ['text',
  'number',
  'number',
  'text',
  'text',
  'text',
  'text',
  'text',
  'number',
  'text',
  'number',
  'number',
  'text',
  'text',
  'number',
  'text

In [129]:
table_names_df, column_names_df, primary_keys_df, foreign_keys_df = create_dfs_from_schema(selected_schema)

In [130]:
table_names_df

,table_names_original,table_names
0,aircraft,aircraft
1,pilot,pilot
2,pilot_record,pilot record


In [131]:
column_names_df

,column_index,table_name,column_names_original,column_names,column_types,is_primary_key
0,0,N/A,*,*,text,False
1,1,aircraft,Aircraft_ID,aircraft id,number,True
2,2,aircraft,Order_Year,order year,number,False
3,3,aircraft,Manufacturer,manufacturer,text,False
4,4,aircraft,Model,model,text,False
5,5,aircraft,Fleet_Series,fleet series,text,False
6,6,aircraft,Powertrain,powertrain,text,False
7,7,aircraft,Fuel_Propulsion,fuel propulsion,text,False
8,8,pilot,Pilot_ID,pilot id,number,True
9,9,pilot,Pilot_name,pilot name,text,False


In [132]:
primary_keys_df

,table_name,primary_key_column_index,primary_key
0,aircraft,1,aircraft id
1,pilot,8,pilot id
2,pilot record,17,pilot id


In [133]:
foreign_keys_df

,foreign_key_column,referenced_column,foreign_key_table,foreign_key_column_name,referenced_table,referenced_column_name
0,18,1,pilot record,aircraft id,aircraft,aircraft id
1,17,8,pilot record,pilot id,pilot,pilot id


In [134]:
schema, eval_foreign_key_maps = load_tables_from_schema_dict(selected_schema)
#print(schema.keys())
#print(eval_foreign_key_maps) 

In [135]:
dataset = registry.construct('dataset_infer', {
   "name": "spider", "schemas": schema, "eval_foreign_key_maps": eval_foreign_key_maps, 
    "db_path": db_path
})

In [136]:
for _, schema_info in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema_info)

In [137]:
spider_schema = dataset.schemas[db_id]

In [138]:
# question = "Who was the pilot who flew first?"

# code = infer(question, spider_schema)
# print(code)

SELECT pilot.Pilot_name FROM pilot JOIN pilot_record ORDER BY pilot_record.Date Desc LIMIT 1


In [150]:
try:
    while True:
        question = input("Enter your question (or type 'exit' to quit): ")
        if question.lower() == "exit":
            print("Exiting...")
            break
        
        # Assuming infer() function is defined somewhere
        code = infer(question, spider_schema)
        
        printmd(f"**{code}**")  # Bold formatting
        print('-----------------')
    
except KeyboardInterrupt:
    print("\nKeyboard interrupt received. Exiting...")

Enter your question (or type 'exit' to quit):  Who was the pilot who flew first?


**SELECT pilot.Pilot_name FROM pilot JOIN pilot_record ORDER BY pilot_record.Date Desc LIMIT 1**

-----------------


Enter your question (or type 'exit' to quit):  Who was the pilot who flew first?


**SELECT pilot.Pilot_name FROM pilot JOIN pilot_record ORDER BY pilot_record.Date Desc LIMIT 1**

-----------------


Enter your question (or type 'exit' to quit):  exit


Exiting...
